In [1]:
import requests as r
from bs4 import BeautifulSoup as bs
import re
import json
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, wait
import concurrent.futures
import jieba
import pandas as pd

In [2]:
def stop_watch():
    global START  
    END = datetime.now()
    timeSpent = str(END - START) 

    START = END
    return timeSpent

In [19]:
def get_urls(page):
    urls_lists = []
    URL = "https://www.wetalk.tw/forum.php?mod=forumdisplay&fid=2&page={}"    
    resp = r.get(URL.format(page))
        
    soup = bs(resp.text, "html5lib")
    for url in range(2,len(soup.select('tbody > tr > th > a.xst'))):        
        urls = soup.select('tbody > tr > th > a.xst')[url]['href']
        urls_lists.append(urls)
    get_forum_contexts(urls_lists)

In [20]:
def get_forum_contexts(urls_lists):
    cont_comm = []   
    for x in range(0,len(urls_lists)):            
        resp_c = r.get(urls_lists[x])
        soup_c = bs(resp_c.text, 'html5lib')          
        news = {}
        news["content"]  = soup_c.select("td.t_f > article")[0].text.replace('\n','').replace('發表','').replace('文字轉載自 新公民議會','').replace('上傳下載附件','').replace('姬小編※※ 如果發現您的留言太長，不妨考慮投稿新公民議會，如獲刊登將有稿費喔!','').replace('圖片來源','').replace('胡嚴編輯','').strip()
        news["reply"]= soup_c.select("span.xg1 > span.xi1")[1].text.replace('\n','')
        cont_comm.append(news)
        save_forum_contexts(cont_comm)

In [21]:
def save_forum_contexts(cont_comm):
    with open("./2018test1.json", 'a',encoding ='UTF-8') as f:
        f.write(json.dumps(cont_comm))

In [22]:
def main(THREAD_NUM):
    with concurrent.futures.ThreadPoolExecutor(THREAD_NUM) as executor:
        executor.map(get_urls, range(1,312))
        
if __name__=="__main__":
    START = datetime.now()
    main(10)
    print('main =' + stop_watch())

In [4]:
import pymongo

In [7]:
from pymongo import MongoClient

In [18]:
connection = pymongo.MongoClient('mongodb://localhost:27017/')

In [19]:
forum = connection.forumdb

In [34]:
forumcc = forum.forumcollection

In [ ]:
forumcc.insert(cont_comm)

In [33]:
db.forumcc.find()